In [ ]:
# default_exp metrics

# Metrics

> Definition of the metrics that can be used in training models. See [fastai2 docs](https://dev.fast.ai/metrics) for more information.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export 
from fastai2.metrics import Dice

## Dice coefficient / F1 score wrapper class
see [here](https://stats.stackexchange.com/questions/273537/f1-dice-score-vs-iou) for F1/Dice score discussion.

In [ ]:
#export
class Dice_f1(Dice):
    "Dice coefficient metric for binary target in segmentation"
    def accumulate(self, learn):
        pred,targ = flatten_check(learn.pred.argmax(dim=self.axis), learn.y[0].argmax(dim=self.axis))
        self.inter += (pred*targ).float().sum().item()
        self.union += (pred+targ).float().sum().item()

## Intersection-Over-Union wrapper class

In [ ]:
#export
class Iou(Dice_f1):
    "Implemetation of the IoU (jaccard coefficient) that is lighter in RAM"
    @property
    def value(self): return self.inter/(self.union-self.inter) if self.union > 0 else None

### Example


In [ ]:
learn = Learner(dls, model, metrics=[Iou, Dice_f1])